<a href="https://colab.research.google.com/github/vinayakgautam368/FACE-UNLOCK/blob/main/Resnet_structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [2]:
class BasicBlock(nn.Module):     # THIS IS THE FIRST CONVOLUTION BLOCK
  def __init__(self,inplanes,planes,stride=1,downsample=None):
    super().__init__()
    self.conv1=nn.Conv2d(inplanes,planes,kernel_size=3,padding=1,stride=stride,bias=False)
    self.bn1=nn.BatchNorm2d(planes)
    self.relu=nn.ReLU(inplace=True)
 
    self.conv2=nn.Conv2d(planes,planes,kernel_size=3,padding=1,stride=1,bias=False)
    self.bn2=nn.BatchNorm2d(planes)
    self.downsample=downsample
    self.stride=stride
 
  def forward(self,x):
 
    identity=x
 
    out=self.conv1(x)
    out=self.bn1(out)
    out=self.relu(out)
 
    out=self.conv2(out)
    out=self.bn2(out)
 
    if self.downsample is not None:
      identity=self.downsample(x)
 
    out+=identity
    out=self.relu(out)
      
 
    return out

In [3]:
class ResNet(nn.Module):
  def __init__(self,block,numbers,classes=512):
    super().__init__()
 
    #this is first simple conv block
    self.inplanes=64
    self.conv1=nn.Conv2d(3,self.inplanes,kernel_size=7,stride=2,padding=3,bias=False)    #  if input is (2,3,224,224)    output is (2,64,112,112)
    self.bn1=nn.BatchNorm2d(self.inplanes)
    self.relu=nn.ReLU(inplace=True)
    self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)       #gives shape  (2,64,56,56)
 
    # now 4 layers
    self.layer1=self._make_layer(block,64,numbers[0])
    self.layer2=self._make_layer(block,128,numbers[1],stride=2)
    self.layer3=self._make_layer(block,256,numbers[2],stride=2)
    self.layer4=self._make_layer(block,512,numbers[3],stride=2)
 
    self.avgpool=nn.AdaptiveAvgPool2d((1,1))
    self.fc=nn.Linear(512,classes)
 
  def _make_layer(self,block,planes,blocks,stride=1):
    downsample=None
 
    if stride!=1 or self.inplanes!=planes:
      downsample=nn.Sequential(nn.Conv2d(self.inplanes,planes,1,stride,bias=False),nn.BatchNorm2d(planes))
    layers=[]
 
    layers.append(block(self.inplanes,planes,stride,downsample))
    self.inplanes=planes
 
    for j in range(1,blocks):
      layers.append(block(self.inplanes,planes))
    return nn.Sequential(*layers)
 
 
  def forward(self,x):
    x=self.conv1(x)
    x=self.bn1(x)
    x=self.relu(x)
    x=self.maxpool(x)
    # print(x.shape)
 
    x=self.layer1(x)
    x=self.layer2(x)
    x=self.layer3(x)
    x=self.layer4(x)
 
    x=self.avgpool(x)
    x=torch.flatten(x,1)
 
    x=self.fc(x)
 
 
    return x

In [4]:
def resnet34():
  numbers=[3,4,6,3]
  model=ResNet(BasicBlock,numbers)
  return model
def resnet18():
  numbers=[2,2,2,2]
  model=ResNet(BasicBlock,numbers)
  return model

In [8]:
net=resnet18()
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  